In [7]:
import sqlite3
import re
import pandas as pd
from dateutil import parser

# Path to your CSV file
csv_file = r"C:\Users\Alagu\DATASCIENCE\amazon_india_complete_2015_2025.csv"
df = pd.read_csv(csv_file)
#print(df.dtypes)

#Delivery_charges_Cleaning
# Find unique values in delivery_charges
unique_delivery_charges = df['delivery_charges'].unique()

print("Unique delivery charges:")
print(unique_delivery_charges)

# Replace null/NaN in delivery_charges with 0.0
df['delivery_charges'] = df['delivery_charges'].fillna(0.0)

value_counts = df['delivery_charges'].value_counts()
print(value_counts)

#Customer-age-group-cleaning
# Find unique values in customer_age_group
unique_customer_age_group = df['customer_age_group'].unique()

# Replace null in customer_age_group
df['customer_age_group'] = df['customer_age_group'].fillna(17)

print("customer_age_group:")
print(unique_customer_age_group)

#Festival_name_sale
# Find unique values in festival_name
unique_festival_name = df['festival_name'].unique()

# Replace null in festival_name
df['festival_name'] = df['festival_name'].fillna("Normal_day")

print("festival_name:")
print(unique_festival_name)

#Customer-rating
# Find unique values in customer_rating
unique_customer_rating = df['customer_rating'].unique()

# Convert customer_rating to numeric (coerce errors to NaN)
df['customer_rating'] = pd.to_numeric(df['customer_rating'], errors='coerce')

# Calculate mean (ignores NaN)
mean_rating = df['customer_rating'].mean()
print(f"Mean customer_rating: {mean_rating}")

# Fill NaN values with mean
df['customer_rating'] = df['customer_rating'].fillna(mean_rating)

# 🔹 Show total null values in each column
#print(df.isnull().sum())

#delivery_charges_delete
# Delete the delivery_charges column
df = df.drop(columns=['delivery_charges'])

#subtotal_inr_delete
# Delete the subtotal_inr column
df = df.drop(columns=['subtotal_inr'])

# Remove symbols and commas from original_price_inr
df['original_price_inr'] = df['original_price_inr'].astype(str)  # Ensure string
df['original_price_inr'] = df['original_price_inr'].str.replace(r'[₹,RsINR\s]', '', regex=True)

# Convert to float
df['original_price_inr'] = df['original_price_inr'].astype(float)

# Optional: verify
print(df['original_price_inr'].head())
print(df['original_price_inr'].dtype)  # should show float64

# Verify
print(df.head())

#Customer-city-cleaning
#Mumbai_city
# Define all variants to replace
mumbai_variants = ['Mumbai', 'MUMBAI', 'Mumbai ', 'Bombay', 'mumba']
# Replace all variants with 'Mumbai'
df["customer_city"] = df["customer_city"].replace(mumbai_variants, "Mumbai")

kolkata_variants = ['Kolkata', 'Kolkata ', 'kolkata', 'Calcutta', 'KOLKATA']
# Replace all variants with 'Kolkata'
df["customer_city"] = df["customer_city"].replace(kolkata_variants, "Kolkata")

delhi_variants = ['Delhi', 'New Delhi', 'DELHI', 'Delhi NCR', 'delhi']
# Replace all variants with 'Delhi'
df["customer_city"] = df["customer_city"].replace(delhi_variants, "Delhi")

chennai_variants = ['Chennai', 'Madras', 'CHENNAI', 'chenai', 'Chennai ']
# Replace all variants with 'chennai'
df["customer_city"] = df["customer_city"].replace(chennai_variants, "Chennai")

Bangalore_variants = ['Bangalore', 'BANGALORE',  'Bengalore', 'Bengaluru', 'Banglore']
# Replace all variants with 'Bangalore'
df["customer_city"] = df["customer_city"].replace(Bangalore_variants, "Bangalore")

# Find unique values in customer_city
unique_customer_city = df['customer_city'].unique()
print("curtomer_city:")
print(unique_customer_city)
value_count_city = df['customer_city'].value_counts()
print(value_count_city)

# Unique values in each column
unique_states = df["customer_state"].unique()
unique_tier = df["customer_tier"].unique()
unique_spending_tier = df["customer_spending_tier"].unique()
unique_payment_method = df["payment_method"].unique()
unique_delivery_type = df["delivery_type"].unique()

print("Unique customer_state:", unique_states)
print("Unique customer_tier:", unique_tier)
print("Unique customer_spending_tier:", unique_spending_tier)
print("unique payment_method:", unique_payment_method)
print("unique delivery_type:", unique_delivery_type)

value_payment_method = df['payment_method'].value_counts()
print(value_payment_method)

# Remove unwanted payment methods
df = df[~df['payment_method'].isin(['Wallet', 'BNPL'])]

# Optional: confirm removal
print("✅ Updated payment method counts:")
print(df['payment_method'].value_counts())


print("Delivery type counts:\n", df["delivery_type"].value_counts())
#print("\nCustomer Tire counts:\n", df["customer_tire"].value_counts())
#print("\nCustomer Spending Tier counts:\n", df["customer_spending_tier"].value_counts())

#is_festival_sale column droppped
# Or inplace
df.drop(columns=["is_festival_sale"], inplace=True)

#original_price_inr_cleaning
# Remove '-' symbol from original_price_inr
df["original_price_inr"] = df["original_price_inr"].astype(str).str.replace("-", "", regex=False)

# Convert back to float (if needed)
df["original_price_inr"] = df["original_price_inr"].astype(float)

# Find rows where '-' is present in original_price_inr
#rows_with_dash = df[df["original_price_inr"].astype(str).str.contains("-")]

#print("Rows with '-' in original_price_inr:")
#print(rows_with_dash)

#price_convert_into_whole_number
# Convert float columns to whole numbers (integer)
df["original_price_inr"] = df["original_price_inr"].astype(float).round().astype(int)
df["discount_percent"] = df["discount_percent"].astype(float).round().astype(int)
df["discounted_price_inr"] = df["discounted_price_inr"].astype(float).round().astype(int)
df["final_amount_inr"] = df["final_amount_inr"].astype(float).round().astype(int)

#discounted_price_checking

# Step 1: Calculate expected discounted price
df["calculated_discounted_price"] = (df["original_price_inr"] * (1 - df["discount_percent"] / 100)).round()

# Step 2: Compare with actual discounted price
df["discount_check"] = df["calculated_discounted_price"] == df["discounted_price_inr"]

# Step 3: Calculate variation percentage
df["variation_percent"] = (
    abs(df["calculated_discounted_price"] - df["discounted_price_inr"]) 
    / df["calculated_discounted_price"] * 100
).round(2)

# Step 4: Mark as High if > 5%
df["variation_flag"] = df["variation_percent"].apply(lambda x: "High" if x > 5 else "OK")

# Step 5: Count how many High
high_count = (df["variation_flag"] == "High").sum()

print("Number of High variation rows:", high_count)
print(df[["original_price_inr", "discount_percent", "discounted_price_inr", 
          "calculated_discounted_price", "variation_percent", "variation_flag"]])

# Delete rows where variation_flag is High
df = df[df["variation_flag"] != "High"]

# Reset index after deletion (optional)
df.reset_index(drop=True, inplace=True)

print("Rows after removing High variation:", len(df))

print("Number of rows present:", df.shape[0])

#drop
df.drop(columns=["calculated_discounted_price"], inplace=True)
df.drop(columns=["variation_percent"], inplace=True)
df.drop(columns=["variation_flag"], inplace=True)
df.drop(columns=["discount_check"], inplace=True)

#discounted_price_inr vs original_price_inr checking
# Step 1: Check if discounted_price_inr * quantity = final_amount_inr
df["amount_check"] = df.apply(
    lambda x: "correct" if round(x["discounted_price_inr"] * x["quantity"]) == round(x["final_amount_inr"]) else "wrong",
    axis=1
)

# Step 2: Count how many "wrong"
wrong_count = (df["amount_check"] == "wrong").sum()

print("Number of wrong rows:", wrong_count)
print(df[["discounted_price_inr", "quantity", "final_amount_inr", "amount_check"]])

# Delete rows where amount_check is "wrong"
df = df[df["amount_check"] != "wrong"]

# Reset index after deletion (optional)
df.reset_index(drop=True, inplace=True)

print("Rows after removing wrong amounts:", len(df))

df.drop(columns=["amount_check"], inplace=True)

##Customer_age_group label and delete the null

# Define the mapping
age_group_map = {
    "17": 0,
    "18-25": 1,
    "26-35": 2,
    "36-45": 3,
    "46-55": 4,
    "55+": 5,
    "+55": 5
}

# Ensure customer_age_group is string
df["customer_age_group"] = df["customer_age_group"].astype(str)

# Create new column with numeric labels
df["age_group_label"] = df["customer_age_group"].map(age_group_map)

print(df[["customer_age_group", "age_group_label"]].head())

## Count how many rows have age_group_label 0 (i.e., "17")
count_17 = (df["age_group_label"] == 0).sum()
print("Number of rows with 17 in age_group_label:", count_17)

# Delete rows where age_group_label is 0
df = df[df["age_group_label"] != 0]

# Reset index after deletion (optional)
df.reset_index(drop=True, inplace=True)

print("Rows after removing age_group_label 0:", len(df))

print("Number of rows present:", df.shape[0])

#drop Customer_age_group
df.drop(columns=["customer_age_group"], inplace=True)

#Prime_member_table update
#print(df.dtypes)
unique_prime_member = df["is_prime_member"].unique()
print("Unique prime_member:", unique_prime_member)
print(df["is_prime_member"].value_counts())

# Convert to string, strip whitespace
df['is_prime_member'] = df['is_prime_member'].astype(str).str.strip()

# Remove rows where value is '0' or '1'
df = df[~df['is_prime_member'].isin(['0', '1'])]

Yes_variants = ['True', 'TRUE', 'Yes']
# Replace all variants with 'is_primme_member'
df["is_prime_member"] = df["is_prime_member"].replace(Yes_variants, "Yes")

No_variants = ['False', 'FALSE', 'No']
# Replace all variants with 'is_prime_member'
df["is_prime_member"] = df["is_prime_member"].replace(No_variants, "No")

print("Rows after removing age_group_label 0:", len(df))

#Customer_rating 
#print(df.dtypes)
unique_customer_rating = df["customer_rating"].unique()
print("customer_rating:", unique_customer_rating)
print(df["customer_rating"].value_counts())
df['customer_rating'] = df['customer_rating'].astype(float).round(1)

#Is_prime_eligible 
#print(df.dtypes)
unique_is_prime_eligible = df["is_prime_eligible"].unique()
print("is_prime_eligible:", unique_is_prime_eligible)
print(df["is_prime_eligible"].value_counts())

# Convert to string, strip whitespace
df['is_prime_eligible'] = df['is_prime_eligible'].astype(str).str.strip()

# Remove rows where value is '0' or '1'
df = df[~df['is_prime_eligible'].isin(['0', '1'])]

#Yes_variants = ['True', 'TRUE', 'Yes']
# Replace all variants with 'is_prime_eligible'
df["is_prime_eligible"] = df["is_prime_eligible"].replace(Yes_variants, "Yes")

#No_variants = ['False', 'FALSE', 'No']
# Replace all variants with 'is_prime_eligible'
df["is_prime_eligible"] = df["is_prime_eligible"].replace(No_variants, "No")

print("Rows after removing age_group_label 0:", len(df))

#delivery_days
unique_delivery_days = df["delivery_days"].unique()
print("delivery_days:", unique_delivery_days)
print(df["delivery_days"].value_counts())
# Convert to string and strip whitespace (in case values have spaces)
df['delivery_days'] = df['delivery_days'].astype(str).str.strip()

# Drop rows with unwanted values
df = df[~df['delivery_days'].isin(['-1', '1-2 days', 'Same Day', 'Express', '0'])]
print("Rows after removing age_group_label 0:", len(df))

##############-- order-date-cleaning-----
# Ensure column is string
df['order_date'] = df['order_date'].astype(str)

# Find rows containing "/"
mask = df['order_date'].str.contains("/", na=False)

# Filtered rows
df_with_slash = df[mask]

# Count how many
count_with_slash = mask.sum()

#print("Number of rows with '/' in order_date:", count_with_slash)
#print(df_with_slash)
# Ensure column is string
df['order_date'] = df['order_date'].astype(str)

# Keep only rows that do NOT contain "/"
df = df[~df['order_date'].str.contains("/", na=False)]

print("Rows after removing order_date_clean:", len(df))
def normalize_date(date_str):
    try:
        # If it starts with 4 digits, assume yyyy-mm-dd → leave as is
        if date_str[:4].isdigit() and date_str[4] in ['-', '/']:
            return date_str
        else:
            # Otherwise parse as dd-mm-yyyy
            dt = pd.to_datetime(date_str, dayfirst=True, errors='coerce')
            if pd.isna(dt):
                return date_str  # leave unrecognized
            return dt.strftime("%Y-%m-%d")
    except:
        return date_str

# Apply normalization
df['order_date'] = df['order_date'].apply(normalize_date)
#print(df.dtypes)
#print(df['order_date'].unique())

#####################################finding_duplicate-value---###############

# Ensure the relevant columns are clean
df['customer_id'] = df['customer_id'].astype(str).str.strip()
df['order_date'] = df['order_date'].astype(str).str.strip()
df['product_id'] = df['product_id'].astype(str).str.strip()
df['original_price_inr'] = pd.to_numeric(df['original_price_inr'], errors='coerce')

# Flag duplicates
df['duplicate_flag'] = df.duplicated(subset=['customer_id', 'order_date', 'product_id', 'original_price_inr'], keep=False)

# Convert boolean to 'duplicate' / 'unique'
df['duplicate_flag'] = df['duplicate_flag'].map({True: 'duplicate', False: 'unique'})

# See the result
print(df[['customer_id', 'order_date', 'product_id', 'original_price_inr', 'duplicate_flag']])

# Count duplicates
duplicate_count = (df['duplicate_flag'] == 'duplicate').sum()

# Remove duplicates and keep the first occurrence
df = df.drop_duplicates(subset=['customer_id', 'order_date', 'product_id', 'original_price_inr'], keep='first')

# Reset index (optional)
df = df.reset_index(drop=True)

#drop Duplicate_flag
df.drop(columns=["duplicate_flag"], inplace=True)

print("Rows after duplicate_clean:", len(df))


# Show count of nulls in each column
print("Null count per column:")
print(df.isnull().sum())

# Show total nulls in entire file
print("\nTotal null values:", df.isnull().sum().sum())

# Show rows that contain at least one null
print("\nRows with null values:")
print(df[df.isnull().any(axis=1)])

#outlinear original_price_inr

# Convert column to numeric (in case it has text, ₹, etc.)
df['original_price_inr'] = pd.to_numeric(df['original_price_inr'], errors='coerce')

# Drop null values in the price column
df = df.dropna(subset=['original_price_inr'])

# Calculate IQR (Interquartile Range)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Count outliers before deletion
outlier_count = df[(df['original_price_inr'] < lower_bound) | (df['original_price_inr'] > upper_bound)].shape[0]
print(f"Number of outliers to delete: {outlier_count}")

# Delete outliers
df_cleaned = df[(df['original_price_inr'] >= lower_bound) & (df['original_price_inr'] <= upper_bound)]

# Show summary after deletion
print(f"Rows before deletion: {len(df)}")
print(f"Rows after deletion: {len(df_cleaned)}")
print(f"Deleted rows: {len(df) - len(df_cleaned)}")

#Category
# Find unique values in category
unique_category = df['category'].unique()
print(unique_category)

# Assuming your dataframe is df and column is 'category'
counts = df['category'].value_counts()
print(counts)

# Define all variants
variants = ['Electronic', 'Electronicss', 'ELECTRONICS', 'Electronics & Accessories']

# Replace them all with 'Electronics'
df['category'] = df['category'].replace(variants, 'Electronics')

print(df['category'].value_counts())


output_file = r"C:\Users\Alagu\DATASCIENCE\Cleaned_data\amazon_india_complete_2015_2025_cleaned_03.csv"
df.to_csv(output_file, index=False)
print(f"✅ Cleaned file saved at: {output_file}")
print("✅ Cleaned file saved as 'cleaned_file.csv'")


Unique delivery charges:
[ 0. nan 40.]
delivery_charges
0.0     1127601
40.0          8
Name: count, dtype: int64
customer_age_group:
['36-45' nan '46-55' '18-25' '26-35' '55+']
festival_name:
[nan 'Summer Sale' 'Back to School' 'Diwali Sale' 'Holi Festival'
 'Amazon Great Indian Festival' 'Republic Day Sale' 'Valentine Sale'
 'Prime Day']
Mean customer_rating: 4.316058126956314
0     27340.84
1     32907.49
2     47052.18
3    238725.44
4     25970.76
Name: original_price_inr, dtype: float64
float64
      transaction_id  order_date         customer_id   product_id  \
0  TXN_2023_00063013  2023-07-23  CUST_2023_00018393  PROD_000454   
1  TXN_2021_00064486  20-07-2021  CUST_2015_00002865  PROD_000579   
2  TXN_2017_00065617  2017-11-16  CUST_2016_00004057  PROD_000295   
3  TXN_2020_00054393  2020-05-04  CUST_2020_00014574  PROD_001654   
4  TXN_2018_00071646  2018-10-09  CUST_2018_00006275  PROD_000095   

                        product_name     category  subcategory      brand  \
0 